# 使用win32模块发邮件

# 1. 导入需要用到的模块

In [1]:
from openpyxl import load_workbook    #openpyxl模块
from email.mime.text import MIMEText  # 邮件正文
from email.header import Header       # 邮件头
import smtplib, time 
from win32com import client

# 2. 读取excel文件和编辑邮件内容函数(使用win32模块)

In [2]:
def op_excel(filename:str):
    # 加载excel文件 data_only=True，自动计算公式
    wb = load_workbook(filename, data_only=True)  # 使用xlsx文件，不要使用xls
    sheet = wb.active
 
    # 先取列名，因为使用了break函数，所以只能取到第一行即列名
    table_col_name = []
    for row in sheet.rows:
        for cell in row:
            # print(cell.value)
            table_col_name.append(cell.value)
        break
 
    # 循环excel
    for row in sheet.iter_rows(min_row = 2):  #min_row表示从第二行开始取值，因为第一行是列名，min
        table_col_html = '<thead>'  # 表头
        row_text = '<tr>'  # 开始一行
        list_del = []  # 准备减掉的字段
        list_finally = table_col_name[2:]  # 最终的字段
        
        #判断是否有值为0，或者为空的字段，如果有就去掉这一列
        for cell in row[2:]:
            # print(cell.value, type(cell.value))    # None <class 'NoneType'>  0 <class 'int'>
            if cell.value == None or cell.value == 0:  # 数据为零或空
                # print(cell.col_idx)   # 列数，从1开始数
                list_del.append(table_col_name[cell.col_idx - 1])
            else:
                row_text += f'<td>{cell.value}</td>'
        row_text += '</tr>'  # 结束一行
 
        # 去掉值为零或空的字段
        for i in list_del:
            list_finally.remove(i)
 
        # 重写首行
        for i in list_finally:
            table_col_html += f'<th>{i}</th>'
        table_col_html += '</thead>'
 
        # 员工姓名
        name = row[2].value  # 从0开始数，第2列
        staff_email = row[1].value  # 员工邮箱地址 从0开始数，第1列
 
        mail_body_context = f'''<h3>{name}：你好！</h3>
            <p>请查收你2020年5月的工资条。</p>
            <table border="1px solid black">
            {table_col_html}
            {row_text}
            </table>
            '''
        # 发邮件
        if staff_email is None:
            continue
        win32_send(staff_email,mail_body_context)             #发送邮件到对应人邮箱
        print(f'成功发送工资条到{staff_email}--{name}...')    #打印出来当前的发送信息
       

# 3. Win32模块发送邮件函数

In [3]:
def win32_send(Receiver,Body):
    outlook = client.Dispatch('outlook.application')  #实例化一个win32对象，对象操纵outlook这个app
    mail = outlook.CreateItem(0)                               #新建一个邮件
    mail.To = Receiver                                         #收件人邮箱
    mail.Subject = '2020年5月工资条'                           #邮件主题
    
    mail.BodyFormat = 2                                        #邮件的文本格式，2代表html格式
    mail.HTMLBody= Body                                        #邮件的正文
    mail.Send()                                                #发送邮件

# 发送邮件吧！

In [4]:
time_start = time.time()  #得到程序开始的时间
filename = 'C:/Users/wangzhuo2/Desktop/工资表.xlsx'       #工资条的表格地址
op_excel(filename)        #读取excel文件中的内容，并发送邮件 
time_end = time.time()    #得到执行结束的时间
print('脚本执行完毕，耗时%.3f秒。' % (time_end - time_start))

成功发送工资条到wangzhuo2@csvw.com--刘明...
脚本执行完毕，耗时0.064秒。


# 使用smtplib模块发邮件

# 1. 登录邮箱

In [ ]:
# 登录邮箱，并返回smtp
def login(smtp_server, ssl_port, sender, passwd):
    smtp_obj = smtplib.SMTP_SSL(smtp_server, ssl_port)
    smtp_obj.login(sender, passwd)
    return smtp_obj

# 2.读取excel文件，并发送邮件

In [ ]:
def op_excel(smtp_obj, filename:str):
    # 加载excel文件 data_only=True，自动计算公式
    wb = load_workbook(filename, data_only=True)  # 使用xlsx文件，不要使用xls
    sheet = wb.active
 
    # 先取列名
    table_col_name = []
    for row in sheet.rows:
        for cell in row:
            # print(cell.value)
            table_col_name.append(cell.value)
        break
 
    # 循环excel，每次循环发送一封邮件
    for row in sheet.iter_rows(min_row = 2):  #从第二行第三列开始，每行都跳过前两列
        table_col_html = '<thead>'  # 表头
        row_text = '<tr>'  # 开始一行
        list_del = []  # 准备减掉的字段
        list_finally = table_col_name[2:]  # 最终的字段
        
        #判断是否有值为0，或者为空的字段，如果有就去掉这一列
        for cell in row[2:]:
            # print(cell.value, type(cell.value))    # None <class 'NoneType'>  0 <class 'int'>
            if cell.value == None or cell.value == 0:  # 数据为零或空
                # print(cell.col_idx)                    # 列数，从1开始数
                list_del.append(table_col_name[cell.col_idx - 1])  #因为列数cell.col_idx是从1开始的，list索引是从0开始的，所以需要减1
            else:
                row_text += f'<td>{cell.value}</td>'
        row_text += '</tr>'  # 结束一行
 
        # 去掉值为零或空的字段
        for i in list_del:
            list_finally.remove(i)
 
        # 重写首行
        for i in list_finally:
            table_col_html += f'<th>{i}</th>'
        table_col_html += '</thead>'
 
        # 员工姓名
        name = row[2].value  # 从0开始数，第2列
        staff_email = row[1].value  # 员工邮箱地址 从0开始数，第1列
 
        mail_body_context = f'''<h3>{name}：你好！</h3>
            <p>请查收你2020年5月的工资条。</p>
            <table border="1px solid black">
            {table_col_html}
            {row_text}
            </table>
            '''
        msg = MIMEText(mail_body_context, 'html', 'utf-8')
 
        msg['From'] = Header('人力资源部', 'utf-8')           # 发送者
        msg['To'] = Header('员工', 'utf-8')                  # 接收者
        msg['Subject'] = Header('2020年5月工资条', 'utf-8')  # 主题
 
        # 发邮件
        try:
            smtp_obj.sendmail(sender, [staff_email], msg.as_string())
            print(f'成功发送工资条到{staff_email}--{name}...')
        except smtplib.SMTPException as e:
            print('Error：无法发送邮件.Case:%s' % e)
 

# 再发一遍！

In [ ]:
time_start = time.time()

smtp_server = 'smtp.qq.com'  # SMTP服务器
ssl_port = 465               # SMTP端口
sender = '1744577940@qq.com'  # 发件人邮箱
passwd = 'pflstiewztbyeegb'  # 发件人密码
filename = input("请输入文件地址：")
# filename = 'C:/Users/wangzhuo2/Desktop/new.xlsx'

smtp_obj = login(smtp_server, ssl_port, sender, passwd)
op_excel(smtp_obj, filename)
time_end = time.time()
print('脚本执行完毕，耗时%.3f秒。' % (time_end - time_start))